# Tarea 2
## IIC2440 - Procesamiento de Datos Masivos

integrantes:
- Rodrigo Nahum
- Fernando Quintana

## Parte 2: Single Source Shortest Path

# Setup

Primero, instalamos pyspark.

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=48a3947731e7ca0c71fe33000c48d4bd79a00be481f6216a1786dd85c30e8d2f
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Luego, importamos PySpark y otras librerias a usar, y cramos un Spark Context.

In [2]:
from pyspark.sql import SparkSession
from itertools import permutations
import random
import json
import math

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext

# Definición del grafo

En esta parte, definimos el grafo a usar. Primero, usamos el grafo dado de ejemplo en la tarea.

In [3]:
node_list = [1, 2, 3, 4]
edges_list = [(1, 2, 10), (2, 3, 3), (2, 4, 24), (3, 2, 1), (2, 1, 5), (4, 3, 7)]

nodes = sc.parallelize(node_list)
edges = sc.parallelize(edges_list).map(lambda x: (x[0], (x[1], x[2])))

Tambien, damos la opción de generar un grafo aleatorio. Abajo, `n` es la cantidad de nodos del grafo, y `edge_chance` es la probabilidad de que una arista cualquiera aparezca en el grafo. O sea, generamos todos los pares de nodos, y para cada uno, decidimos si esa arista está en el grafo, según la probabilidad de `edge_chance`.

In [ ]:
n = 10000

edge_chance = 0.0002

nodes_list = [i + 1 for i in range(n)]

edges_list = []
for edge in permutations(nodes_list, 2):
    if random.random() < edge_chance:
        edge_cost = random.randint(20, 40)
        edges_list.append([*edge, edge_cost])


nodes = sc.parallelize(nodes_list)
edges = sc.parallelize(edges_list).map(lambda x: (x[0], (x[1], x[2])))

# Ver si ponemos como ejemplo el grafo de cora

# Single Source Shortest Path

## Inicialización

Para implementar este algoritmo, primero necesitamos definir el nodo de inicio. A modo de ejemplo, usaremos el nodo 1 como inicial. Luego, inicializamos los estados de los nodos. En este algoritmo, todos los nodos parten con costo infinito, excepto el nodo inicial, que parte con costo 0.

In [4]:
initial_node = 1
current_cost = nodes.map(lambda x: (x, 0 if x == initial_node else float('inf')))

## Generación de mensajes.

Luego, debemos crear los mensajes que cada nodo enviará. Para esto, hacemos un join entre los valores actuales y las aristas. Notemos que, en ambos RDDs, la llave es el nodo de origen. De esta forma, obtenemos tuplas que tienen el costo del nodo de origen, y el costo de una arista que sale de ese nodo (además del nodo de destino). Luego, hacemos un map para reordenar la información y sumar el costo del nodo con el de la arista. Obtenemos nuevas tuplas, en donde la llave es el nodo de destino, y el valor es el costo del nodo de origen más el de la arista.

In [5]:
messages = current_cost.join(edges, 4).map(lambda x: (x[1][1][0], (x[1][0] + x[1][1][1])))

## Filtrado de mensajes.

En este caso, sí queremos filtrar mensajes. Los mensajes con costo infinito no nos interesan, ya que no aportan información a los nodos receptores. Entonces, filtramos todos los mensajes con costo infinito.

In [7]:
filtered_messages =  messages.filter(lambda x: x[1] != float('inf'))

## Agregación de mensajes.

Luego, definimos una función para agregar los mensajes recibidos por cada nodo. Para esto, usamos un reduceByKey, de forma de juntar todos los mensajes enviados a un nodo (recordemos que, en los mensajes, la llave es el nodo de destino), y reducirlos según nuestra función de agregación.
En este caso, solo queremos quedarnos con el mínimo costo recibido. Por lo tanto, la función de agregación será simplemente la función `min` de Python.

In [ ]:
aggregated_messages = filtered_messages.reduceByKey(lambda x, y: min(x, y))

## Update del estado

Luego, para updatear el estado de los nodos, debemos obtener el estado actual y el resultado de agregar los mensajes. Para esto, usamos un leftOuterJoin entre los estados actuales, y los mensajes agregados (de nuevo, recordemos que la llave de los mensajes agregados es el nodo receptor, entonces, al hacer el leftOuterJoin, tendremos el estado del nodo receptor y el resultado de agregar sus mensajes recibidos). Hacemos un leftOuterJoin, ya que pueden existir nodos sin aristas entrantes, o sea, que no recibieron ningún mensaje.

Luego del leftOuterJoin, hacemos un map para reordenar la información y computar el nuevo valor del nodo. En este caso, de nuevo, nos queremos quedar con el mínimo entre el estado actual y el resultado de agregar los mensajes. //Poner eso de que checkeamos que es none porque puede ser que el nodo no reciba ningun mensaje.

In [ ]:
def compare_values(tup):
  if (tup[1]) is None:
    return tup[0]
  return min(tup)

def get_diff(tup):
  if tup[1][0] == float('inf') and tup[1][1] == float('inf'):
    return 0
  return tup[1][1] - tup[1][0]

def single_source_shortest_path(nodes, edges, initial_node):
    current_cost = nodes.map(lambda x: (x, 0 if x == initial_node else float('inf')))

    partitions = math.ceil(nodes.count() / 1000)

    stop = False
    old_values = current_cost
    old_values.checkpoint()

    iter = 0
    while not stop:
        print(f"At iter {iter}")
        messages = old_values.join(edges, partitions).map(lambda x: (x[1][1][0], (x[1][0] + x[1][1][1])))
        new_cost = old_values.leftOuterJoin(messages.reduceByKey(lambda x, y: min(x, y)), partitions).map(lambda x: (x[0], compare_values(x[1])))
        # messages = old_values.join(edges, partitions).map(lambda x: (x[1][1][0], (x[1][0] + x[1][1][1]))).union(old_values)
        # new_cost = messages.reduceByKey(lambda x, y: min(x, y))
        new_cost.checkpoint()
        max_change = new_cost.join(old_values, partitions).map(get_diff).max()
        if max_change == 0:
            stop = True
        else:
            old_values = new_cost
            iter += 1

    return new_cost.collect()


In [ ]:
shortest_paths = single_source_shortest_path(nodes, edges, 1)

At iter 0
At iter 1
At iter 2
At iter 3
At iter 4
At iter 5
At iter 6
At iter 7


In [ ]:
print(shortest_paths.toDebugString().decode('utf-8'))

(10) PythonRDD[8203] at RDD at PythonRDD.scala:53 []
 |   ReliableCheckpointRDD[8205] at count at <ipython-input-246-c131b016c7ed>:29 []


In [ ]:
shortest_paths.collect()

[(10, 130),
 (20, 142),
 (30, 123),
 (40, 147),
 (50, 84),
 (60, 118),
 (70, 101),
 (80, 122),
 (90, 141),
 (100, 101),
 (110, 85),
 (120, 129),
 (130, 109),
 (140, 92),
 (150, 129),
 (160, 121),
 (170, 108),
 (180, 113),
 (190, 104),
 (200, 123),
 (210, 137),
 (220, 109),
 (230, 132),
 (240, 137),
 (250, 120),
 (260, 133),
 (270, 123),
 (280, 74),
 (290, 134),
 (300, 133),
 (310, 113),
 (320, 119),
 (330, 122),
 (340, 91),
 (350, 100),
 (360, 116),
 (370, 139),
 (380, 96),
 (390, 123),
 (400, 132),
 (410, 116),
 (420, 129),
 (430, 85),
 (440, 127),
 (450, 120),
 (460, 127),
 (470, 116),
 (480, 138),
 (490, 122),
 (500, 113),
 (510, 108),
 (520, 129),
 (530, 133),
 (540, 134),
 (550, 126),
 (560, 129),
 (570, 132),
 (580, 129),
 (590, 128),
 (600, 120),
 (610, 122),
 (620, 101),
 (630, 101),
 (640, 135),
 (650, 132),
 (660, 99),
 (670, 133),
 (680, 110),
 (690, 116),
 (700, 105),
 (710, 126),
 (720, 134),
 (730, 124),
 (740, 100),
 (750, 138),
 (760, 121),
 (770, 124),
 (780, 153),
 (7